# Lesson 6 HW

1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [299]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [300]:
from sklearn.datasets import make_classification

In [301]:
# создадим датасет
X, y = make_classification(n_samples=10000, n_features=7, n_informative=7, n_classes=2, n_redundant=0, scale=10)

In [302]:
X = pd.DataFrame(X)
y = pd.Series(y)

In [303]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   stratify=y, random_state=15)

2. Обучить любой классификатор (какой вам нравится)

In [304]:
from lightgbm import LGBMClassifier

In [305]:
model = LGBMClassifier(random_state=5)

In [306]:
model.fit(X_train, y_train)

LGBMClassifier(random_state=5)

In [307]:
y_score = model.predict_proba(X_test)[:, 1]

In [308]:
from sklearn.metrics import roc_auc_score, precision_recall_curve

In [309]:
results = pd.DataFrame(columns=['method', 'fscore', 'roc_auc', 'precision', 'recall'])

In [310]:
b = 1
precision, recall, thresholds = precision_recall_curve(y_test, y_score)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
ix = np.nanargmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.49845019328774154, F-Score=0.943, Precision=0.953, Recall=0.933


In [311]:
results = results.append({'method': 'usual',
                'fscore': fscore[ix],
                'roc_auc': roc_auc_score(y_test, y_score),
                'precision': precision[ix],
                'recall': recall[ix]}, ignore_index=True)

3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть

In [312]:
mod_data = X_train.copy()
mod_data['label'] = y_train
mod_data.reset_index(drop=True, inplace=True)

In [313]:
pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]

np.random.shuffle(pos_ind)

perc = 0.2
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [314]:
mod_data['class_test'] = -1

In [315]:
mod_data.loc[pos_sample, 'class_test'] = 1

In [316]:
mod_data['class_test'].value_counts()

-1    6300
 1     700
Name: class_test, dtype: int64

4. Применить random negative sampling для построения классификатора в новых условиях

In [317]:
mod_data = mod_data.sample(frac=1)

In [318]:
data_N = mod_data[mod_data['class_test'] == - 1]
data_P = mod_data[mod_data['class_test'] == 1]

In [319]:
neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

In [320]:
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [321]:
sample_train.tail(2)

,0,1,2,3,4,5,6,label,class_test
1135,-2.203329,-24.160735,10.052355,-6.598345,-6.904621,-8.680171,-8.641505,1,-1
3220,0.969711,-8.917024,32.774386,1.581046,8.235981,-27.675876,-3.380534,1,1


In [322]:
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

In [323]:
model.fit(sample_train.drop(columns=['class_test', 'label']),
         sample_train['class_test'])

y_score = model.predict_proba(X_test)[:, 1]

In [324]:
b = 1
precision, recall, thresholds = precision_recall_curve(y_test, y_score)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
ix = np.nanargmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.23037748715942147, F-Score=0.868, Precision=0.832, Recall=0.907


5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик)

In [325]:
results = results.append({'method': 'Random Negative Sampling',
                'fscore': fscore[ix],
                'roc_auc': roc_auc_score(y_test, y_score),
                'precision': precision[ix],
                'recall': recall[ix]}, ignore_index=True)

In [326]:
results

,method,fscore,roc_auc,precision,recall
0,usual,0.943078,0.985165,0.953029,0.933333
1,Random Negative Sampling,0.867985,0.934455,0.831907,0.907333


In [327]:
results.drop(columns='method').iloc[0] - results.drop(columns='method').iloc[1]

fscore       0.075094
roc_auc      0.050711
precision    0.121122
recall       0.026000
dtype: float64

Основные потери произошли в точности из-за небольшого количества размеченных данных (20% от всего количества).